In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
from os import walk
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn import preprocessing
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pickle
import h5py

Loading the dataset and normalizing

In [ ]:
test_input = pd.read_pickle('DATA/test_input')
test_output = pd.read_pickle('DATA/test_output')
index = test_input.columns

# Normlaization of input data
# Data normalization according to training dataset/ model
filehandler = open('Weights/Norm', 'rb') 
std_scaler = pickle.load(filehandler)
inputn = pd.DataFrame(std_scaler.transform(test_input), columns=test_input.columns) 
# inputn is still a daraframe with numeric index
outputn = test_output/10**6  #  change of units, outputn is still a daraframe with time index

Define the models to be tested

In [ ]:
include_wave = 'no'
modelID = 3
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 

# Retrive features based on the modelID
index1 = pd.core.indexes.base.Index([]) # create a blank index array
if include_wave == 'yes': 
    index1 = index1.append(index[0:3])
if modelID == 1: # Acc17
    index1 = index1.append(index[[3,4,9,10,15,16]])
if modelID == 2: # Acc38
    index1 = index1.append(index[[5,6,11,12,17,18]])
if modelID == 3: # Acc77
    index1 = index1.append(index[[7,8,13,14,19,20]])
if modelID == 4: # Acc17&38
    index1 = index1.append(index[[3,4,5,6,9,10,11,12,15,16,17,18]])   
if modelID == 5: # Acc17&38&77
    index1 = index1.append(index[3:21])

index1 = index1.append(index[21:]) # SCADA
X = inputn[index1].values
Y = outputn.values

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
def NLL(y, distr): 
  return -distr.log_prob(y) 

def normal_sp(params): 
  return tfp.distributions.Normal(loc=params[:,0:2], scale=1e-3 
                                  + tf.math.softplus(0.05 * params[:,2:4]))# both parameters are learnable

kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X.shape[0] * 1.0)
bias_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X.shape[0] * 1.0)

inputs = tf.keras.layers.Input(shape=(X.shape[1],))

hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(inputs)
hidden = tfp.layers.DenseFlipout(64,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
params = tfp.layers.DenseFlipout(4,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn)(hidden)
dist = tfp.layers.DistributionLambda(normal_sp)(params)


model = Model(inputs=inputs, outputs=dist)
model.compile(Adam(learning_rate=0.0002), loss=NLL) 

model_params = Model(inputs=inputs, outputs=params)
model.summary()

In [ ]:
def compute_predictions_pbnn(model, samples):
    prediction_distribution= model(samples)
    prediction_mean = np.squeeze(prediction_distribution.mean().numpy())/10
    prediction_stdv = np.squeeze(prediction_distribution.stddev().numpy())/10

    # The 95% CI is computed as mean ± (1.96 * stdv)
    upper = (prediction_mean + (1.96 * prediction_stdv))
    lower = (prediction_mean - (1.96 * prediction_stdv))

    return prediction_mean, prediction_stdv, upper, lower

def loglikelihood(y, loc, scale):
    dist = tfp.distributions.Normal(loc, scale)
    return dist.log_prob(y)


In [ ]:
durations = ['3M','6M','9M','12M','15M','18M','21M','24M']
# durations = ['6M','12M','18M','24M']

PU_Mtl = np.zeros([len(durations),len(X)])
PU_Mtn = np.zeros([len(durations),len(X)])
LL_Mtl = np.zeros([len(durations),len(X)])
LL_Mtn = np.zeros([len(durations),len(X)])

for ind in range(len(durations)):
    DEM_tl = np.zeros([len(X)])
    DEM_tn = np.zeros([len(X)])
    DEM_tl2 = np.zeros([len(X)])
    DEM_tn2 = np.zeros([len(X)])
    ll_Mtl = np.zeros([len(X)])
    ll_Mtn = np.zeros([len(X)])
    file = h5py.File('Weights/02_PredictionModel1/Model%d_IncWave%s_%s.h5' % (modelID, include_wave, durations[ind]), 'r')
    weight = []
    for i in range(len(file.keys())):
        weight.append(file['weight' + str(i)][:])
    model.set_weights(weight)
    file.close()
    nsim = 1000
    for j in range(nsim):
#         np.random.seed(j)
        dems = model.predict(X)
        DEM_tl += dems[:,0]
        DEM_tn += dems[:,1]
        DEM_tl2 += (dems[:,0])**2
        DEM_tn2 += (dems[:,1])**2
        prediction_mean, prediction_stdv, upper, lower = compute_predictions_pbnn(model, X)
        lls = loglikelihood(Y, prediction_mean, prediction_stdv)
        ll_Mtl += lls[:,0]
        ll_Mtn += lls[:,1]

    PU_Mtl[ind, :] = np.sqrt((DEM_tl2/nsim)-(DEM_tl/nsim)**2)/(DEM_tl/nsim)
    PU_Mtn[ind, :] = np.sqrt((DEM_tn2/nsim)-(DEM_tn/nsim)**2)/(DEM_tn/nsim)
#     print(PU_Mtl)
#     print(PU_Mtn)
    LL_Mtl[ind,:] = ll_Mtl/nsim
    LL_Mtn[ind,:] = ll_Mtn/nsim
#     print(LL_Mtl)
#     print(LL_Mtn)
    print(ind)
    

In [ ]:
TestLL_Mtl = np.mean(LL_Mtl, axis = 1)
TestLL_Mtn = np.mean(LL_Mtn, axis = 1)
TestPU_Mtl = np.mean(PU_Mtl, axis = 1)
TestPU_Mtn = np.mean(PU_Mtn, axis = 1)

# To plot x-axis according to dataset size
durations = ['3M','6M','9M','12M','15M','18M','21M','24M']
# durations = ['6M','12M','18M','24M']
train_end_dates = ['2018-03-31 23:50:00+00:00', '2018-06-30 23:50:00+00:00', '2018-09-30 23:50:00+00:00', 
                  '2018-12-31 23:50:00+00:00', '2019-03-31 23:50:00+00:00', '2019-06-30 23:50:00+00:00',
                  '2019-09-30 23:50:00+00:00', '2019-12-31 23:50:00+00:00']
ticks = np.zeros([len(durations)])
train_input = pd.read_pickle('DATA/train_input')
for i in range(len(durations)):   
    input = train_input.loc['2018-01-01 00:00:00+00:00':train_end_dates[i]]
    ticks[i] = len(input)

cm = 1/2.54  # centimeters in inches
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9
fig, ax = plt.subplots(1, figsize=(8.5*cm, 6*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.21, right=.98, top=0.98, bottom=0.25, hspace = 0.65, wspace=0.15)
ax.plot(ticks, TestPU_Mtl, color = '#2171b5', marker = 'o', markersize = 5, ls =':', label=r'DEM$_{tl}$')
ax.plot(ticks, TestPU_Mtn,  color = '#08306b', marker = 'o', markersize = 5, label=r'DEM$_{tn}$')

ax.legend(loc = 'upper right')
ax.set_xticks(ticks)
ax.set_xticklabels(durations, rotation= 90)
ax.set_xlabel('Duration of data collection') 
ax.set_ylabel(r'CoV($\hat{DEM}$)') 
plt.grid(axis='y', color='0.95')
fig.savefig('Figures/03_PredictionModel/CoV_vs_Trainsize.pdf')

fig, ax = plt.subplots(1, figsize=(8.5*cm, 6*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.21, right=.98, top=0.98, bottom=0.25, hspace = 0.65, wspace=0.15)
ax.plot(ticks, TestLL_Mtl , color = '#2171b5', marker = 'o', markersize = 5, ls =':', label=r'DEM$_{tl}$')
ax.plot(ticks, TestLL_Mtn ,  color = '#08306b', marker = 'o', markersize = 5, label=r'DEM$_{tn}$')
ax.legend(loc = 'upper left')
ax.set_xticks(ticks)
ax.set_xticklabels(durations, rotation= 90)
ax.set_xlabel('Duration of data collection')  
ax.set_ylabel(r'$\mathbf{\mathbb{E}}$ $[\mathbf{\mathcal{L}}(DEM)]$')  
plt.grid(axis='y', color='0.95')
fig.savefig('Figures/03_PredictionModel/LL_vs_Trainsize.pdf')